In [54]:
import torch
import h5py
import os
import numpy as np
import pandas as pd
from dataclasses import dataclass, field
import torch_geometric
from torch_geometric.data import Data, DataLoader
from dataclasses import dataclass
import utils
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from scipy.signal import resample
import networkx as nx
from joblib import Parallel, delayed
import yaml
import fnmatch
from datetime import datetime
from mne_features.univariate import (
    compute_variance,
    compute_hjorth_complexity,
    compute_hjorth_mobility,
    compute_line_length,
    compute_higuchi_fd,
    compute_katz_fd,
    compute_energy_freq_bands
)
from joblib import Parallel, delayed
import gc
import multiprocessing as mp


In [2]:
@dataclass
class HDFDataset_Writer:
    seizure_lookback: int = 600
    sample_timestep: int = 5
    inter_overlap: int = 0
    preictal_overlap: int = 0
    ictal_overlap: int = 0
    downsample: int = None
    sampling_f: int = 256
    self_loops: bool = False
    balance: bool = False
    smote: bool = False
    buffer_time: int = 15
    connectivity_metric : str = "plv"
    npy_dataset_path: str = "npy_dataset"
    event_tables_path: str = "event_tables"
    cache_folder: str = "cache"
    
    
    def _find_matching_configs(self, current_config):
        
        def find_yaml_files(directory):
            yaml_files = []
            for root, dirs, files in os.walk(directory):
                for file in files:
                    if fnmatch.fnmatch(file, "*.yaml"):
                        yaml_files.append(os.path.join(root, file))
            return yaml_files
        
        config_files = find_yaml_files(self.cache_folder)
        for config_file in config_files:
            with open(config_file) as f:
                config_dict = yaml.load(f, Loader=yaml.FullLoader)
                if config_dict == current_config:
                    print(f"Found matching config file {config_file}")
                    return True
        return False
        
    
    def _create_config_dict(self):
        dataclass_keys = list(self.__dataclass_fields__.keys())
        dict_values = [self.__getattribute__(key) for key in dataclass_keys]
        initial_config_dict = dict(zip(dataclass_keys, dict_values))
        return initial_config_dict
    
    def _create_config_file(self,config_dict, dataset_folder_path):
        with open(os.path.join(dataset_folder_path, "config.yaml"), "w") as f:
            yaml.dump(config_dict, f)
    
    def _get_event_tables(self, patient_name: str) -> tuple[dict, dict]:
        """Read events for given patient into start and stop times lists from .csv extracted files.
        Args:
            patient_name: (str) Name of the patient to get events for.
        Returns:
            start_events_dict: (dict) Dictionary with start events for given patient.
            stop_events_dict: (dict) Dictionary with stop events for given patient.
        """

        event_table_list = os.listdir(self.event_tables_path)
        patient_event_tables = [
            os.path.join(self.event_tables_path, ev_table)
            for ev_table in event_table_list
            if patient_name in ev_table
        ]
        patient_event_tables = sorted(patient_event_tables)
        patient_start_table = patient_event_tables[
            0
        ]  ## done terribly, but it has to be so for win/linux compat
        patient_stop_table = patient_event_tables[1]
        start_events_dict = pd.read_csv(patient_start_table).to_dict("index")
        stop_events_dict = pd.read_csv(patient_stop_table).to_dict("index")
        return start_events_dict, stop_events_dict

    def _get_recording_events(self, events_dict, recording) -> list[int]:
        """Read seizure times into list from event_dict.
        Args:
            events_dict: (dict) Dictionary with events for given patient.
            recording: (str) Name of the recording to get events for.
        Returns:
            recording_events: (list) List of seizure event start and stop time for given recording.
        """
        recording_list = list(events_dict[recording + ".edf"].values())
        recording_events = [int(x) for x in recording_list if not np.isnan(x)]
        return recording_events

    def _create_edge_idx_and_attributes(
        self, connectivity_matrix: np.ndarray, threshold: int = 0.0
    ) -> tuple[np.ndarray, np.ndarray]:
        """Create adjacency matrix from connectivity matrix. Edges are created for values above threshold.
        If the edge is created, it has an attribute "weight" with the value of the connectivity measure associated.
        Args:
            connectivity_matrix: (np.ndarray) Array with connectivity values.
            threshold: (float) Threshold for creating edges. (default: 0.0)
        Returns:
            edge_index: (np.ndarray) Array with edge indices.
            edge_weights: (np.ndarray) Array with edge weights.
        """
        result_graph = nx.graph.Graph()
        n_nodes = connectivity_matrix.shape[0]
        result_graph.add_nodes_from(range(n_nodes))
        edge_tuples = [
            (i, j)
            for i in range(n_nodes)
            for j in range(n_nodes)
            if connectivity_matrix[i, j] > threshold
        ]
        result_graph.add_edges_from(edge_tuples)
        edge_index = nx.convert_matrix.to_numpy_array(result_graph)
        # connection_indices = np.where(edge_index==1)
        # edge_weights = connectivity_matrix[connection_indices] ## ??

        return edge_index

    def _features_to_data_list(self, features, edges, labels, edge_weights=None):
        """Converts features, edges and labels to list of torch_geometric.data.Data objects.
        Args:
            features: (np.ndarray) Array with features.
            edges: (np.ndarray) Array with edges.
            labels: (np.ndarray) Array with labels.
        Returns:
            data_list: (list) List of torch_geometric.data.Data objects.
        """
        data_list = [
            Data(
                x=features[i],
                edge_index=edges[i],
                edge_attr=edge_weights[i],
                y=labels[i],
                # time=time_label[i],
            )
            for i in range(len(features))
        ]
        return data_list

    def _apply_smote(self, features, labels):
        """Performs SMOTE oversampling on the dataset. Implemented for preictal vs ictal scenarion only.
        Args:
            features: (np.ndarray) Array with features.
            labels: (np.ndarray) Array with labels.
        Returns:
            x_train_smote: (np.ndarray) Array with SMOTE oversampled features.
            y_train_smote: (np.ndarray) Array with SMOTE oversampled labels.
        """
        dim_1, dim_2, dim_3 = features.shape

        new_dim = dim_1 * dim_2
        new_x_train = features.reshape(new_dim, dim_3)
        new_y_train = []
        for i in range(len(labels)):
            new_y_train.extend([labels[i]] * dim_2)

        new_y_train = np.array(new_y_train)

        # transform the dataset
        oversample = SMOTE(random_state=42)
        x_train, y_train = oversample.fit_resample(new_x_train, new_y_train)
        x_train_smote = x_train.reshape(int(x_train.shape[0] / dim_2), dim_2, dim_3)
        y_train_smote = []
        for i in range(int(x_train.shape[0] / dim_2)):
            # print(i)
            value_list = list(y_train.reshape(int(x_train.shape[0] / dim_2), dim_2)[i])
            # print(list(set(value_list)))
            y_train_smote.extend(list(set(value_list)))
            ## Check: if there is any different value in a list
            if len(set(value_list)) != 1:
                print(
                    "\n\n********* STOP: THERE IS SOMETHING WRONG IN TRAIN ******\n\n"
                )
        y_train_smote = np.array(y_train_smote)
        # print(np.unique(y_train_smote,return_counts=True))
        return x_train_smote, y_train_smote

    
        
    
    def _get_labels_features_edge_weights_seizure(self, patient):
        """Method to extract features, labels and edge weights for seizure and interictal samples."""

        event_tables = self._get_event_tables(
            patient
        )  # extract start and stop of seizure for patient
        patient_path = os.path.join(self.npy_dataset_path, patient)
        recording_list = [
            recording
            for recording in os.listdir(patient_path)
            if "seizures" in recording
        ]
        self.hdf5_file.create_group(patient)
        for n, record in enumerate(
            recording_list
        ):  # iterate over recordings for a patient
            recording_path = os.path.join(patient_path, record)
            record = record.replace(
                "seizures_", ""
            )  ## some magic to get it properly working with event tables
            record_id = record.split(".npy")[0]  #  get record id
            start_event_tables = self._get_recording_events(
                event_tables[0], record_id
            )  # get start events
            stop_event_tables = self._get_recording_events(
                event_tables[1], record_id
            )  # get stop events
            data_array = np.load(recording_path)  # load the recording

            features, labels, time_labels = utils.extract_training_data_and_labels(
                data_array,
                start_event_tables,
                stop_event_tables,
                fs=self.sampling_f,
                seizure_lookback=self.seizure_lookback,
                sample_timestep=self.sample_timestep,
                preictal_overlap=self.preictal_overlap,
                ictal_overlap=self.ictal_overlap,
                buffer_time=self.buffer_time,
            )

            if features is None:
                print(
                    f"Skipping the recording {record} patients {patient} cuz features are none"
                )
                continue

            features = features.squeeze(2)
            sampling_f = self.sampling_f if self.downsample is None else self.downsample
            if self.downsample:
                new_sample_count = int(self.downsample * self.sample_timestep)
                features = resample(features, new_sample_count, axis=2)
                
            conn_matrix_list = [
                self.connectivity_function(feature, sampling_f) for feature in features
            ]
            edge_idx = np.stack(
                [
                    self._create_edge_idx_and_attributes(
                        conn_matrix, threshold=np.mean(conn_matrix)
                    )
                    for conn_matrix in conn_matrix_list
                ]
            )

            
            # if self.smote:
            #     features, labels = self._apply_smote(features, labels)
            labels = labels.reshape((labels.shape[0], 1)).astype(np.float32)

            features_patient = (
                features if n == 0 else np.concatenate([features_patient, features])
            )
            labels_patient = (
                labels if n == 0 else np.concatenate([labels_patient, labels])
            )
            edge_idx_patient = (
                edge_idx if n == 0 else np.concatenate([edge_idx_patient, edge_idx])
            )
        #     edge_weights_patient = edge_weights if n == 0 else np.concatenate([edge_weights_patient, edge_weights])
        try:
            try:
                self.sample_count += features_patient.shape[0]
            except AttributeError:
                self.sample_count = features_patient.shape[0]
                self.n_channels = features_patient.shape[1]
                self.n_features = features_patient.shape[2]
            
            self.hdf5_file[patient].create_dataset("features", data=features_patient,maxshape=(None,self.n_channels,self.n_features))
            self.hdf5_file[patient].create_dataset("labels", data=labels_patient,maxshape=(None,1))
            self.hdf5_file[patient].create_dataset("edge_idx", data=edge_idx_patient,maxshape=(None,self.n_channels,self.n_channels))
            
        except:
            print("##############################################")
            print(f"Cannot create dataset for patient {patient}!")
            print("##############################################")
        try:
            self.preictal_samples_dict[patient] = np.unique(labels_patient, return_counts=True)[1][0]
        except AttributeError:
            self.preictal_samples_dict = {}
            self.preictal_samples_dict[patient] = np.unique(labels_patient, return_counts=True)[1][0]
    def _get_labels_features_edge_weights_interictal(
        self, patient, samples_patient: int = None
    ):
        """Method to extract features, labels and edge weights for interictal samples.
        Args:
            patient: (str) Name of the patient to extract the data for.
            samples_patient (optional): (int) Number of samples to extract for a patient.
        Samples are extracted from non-seizure recordings for a patient, starting from random time point.
        If not specified, the number of samples is calculated as the number of interictal samples for a patient
        divided by the number of recordings for a patient.

        """
        patient_path = os.path.join(self.npy_dataset_path, patient)
        ## get all non-seizure recordings for a patient
        recording_list = [
            recording
            for recording in os.listdir(patient_path)
            if not "seizures_" in recording
        ]
        if samples_patient is None:
            ## if not specified use the same number of samples for each recording as for preictal samples
            samples_per_recording = int(self.preictal_samples_dict[patient] / len(recording_list))
        else:
            samples_per_recording = int(samples_patient / len(recording_list))
            
        for n, record in enumerate(recording_list):
            recording_path = os.path.join(patient_path, record)
            data_array = np.expand_dims(np.load(recording_path), 1)
            try:
                features, labels = utils.extract_training_data_and_labels_interictal(
                    input_array=data_array,
                    samples_per_recording=samples_per_recording,
                    fs=self.sampling_f,
                    timestep=self.sample_timestep,
                    overlap=self.inter_overlap,
                )
            except ValueError:
                print(f"Cannot extract demanded amount of samples from recording {record} for patient {patient}")
                continue
            if features is None:
                print(
                    f"Skipping the recording {record} patients {patient} cuz features are none"
                )
                continue
            
            idx_to_delete = np.where(
                np.array([np.diff(feature, axis=-1).mean() for feature in features])
                == 0
            )[0]
            if len(idx_to_delete) > 0:
                features = np.delete(features, obj=idx_to_delete, axis=0)
                labels = np.delete(labels, obj=idx_to_delete, axis=0)
                print(
                    f"Deleted {len(idx_to_delete)} samples from patient {patient} \n recording {record} due to zero variance"
                )
            features = features.squeeze(2)
            sampling_f = self.sampling_f if self.downsample is None else self.downsample
            if self.downsample:
                new_sample_count = int(self.downsample * self.sample_timestep)
                features = resample(features, new_sample_count, axis=2)
                
            conn_matrix_list = [
                self.connectivity_function(feature, sampling_f) for feature in features
            ]
            edge_idx = np.stack(
                [
                    self._create_edge_idx_and_attributes(
                        conn_matrix, threshold=np.mean(conn_matrix)
                    )
                    for conn_matrix in conn_matrix_list
                ]
            )
     
            labels = labels.reshape((labels.shape[0], 1)).astype(np.float32)

            features_patient = (
                features if n == 0 else np.concatenate([features_patient, features])
            )
            labels_patient = (
                labels if n == 0 else np.concatenate([labels_patient, labels])
            )
            edge_idx_patient = (
                edge_idx if n == 0 else np.concatenate([edge_idx_patient, edge_idx])
            )
        current_patient_features = self.hdf5_file[patient]["features"].shape[0]
        current_patient_labels = self.hdf5_file[patient]["labels"].shape[0]
        current_patient_edge_idx = self.hdf5_file[patient]["edge_idx"].shape[0]
        self.hdf5_file[patient]["features"].resize(
            (current_patient_features + features_patient.shape[0]),axis=0
        )
        self.hdf5_file[patient]["features"][-features_patient.shape[0]:] = features_patient
        self.hdf5_file[patient]["labels"].resize(
            (current_patient_labels + labels_patient.shape[0]),axis=0
        )
        self.hdf5_file[patient]["labels"][-labels_patient.shape[0]:] = labels_patient
        self.hdf5_file[patient]["edge_idx"].resize(
            (current_patient_edge_idx + edge_idx_patient.shape[0]),axis=0
        )
        self.hdf5_file[patient]["edge_idx"][-edge_idx_patient.shape[0]:] = edge_idx_patient
        
        self.sample_count += features_patient.shape[0]
        
    def get_dataset(self):
        
        folder_name = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        dataset_folder = os.path.join(self.cache_folder, folder_name)
        dataset_path = os.path.join(dataset_folder, "dataset.hdf5")
        current_config = self._create_config_dict()
        if self._find_matching_configs(current_config):
            print(
                f"Dataset already exists. Dataset not created."
            )
            return dataset_folder
        
        os.makedirs(dataset_folder, exist_ok=True)
        self._create_config_file(current_config, dataset_folder)
        self.hdf5_file = h5py.File(dataset_path, "w")
        patient_list = os.listdir(self.npy_dataset_path)
        self.connectivity_function = utils.compute_plv_matrix if self.connectivity_metric == "plv" else utils.compute_spect_corr_matrix
        try:
            if self.smote:
                for patient in patient_list:
                    self._get_labels_features_edge_weights_seizure(patient)
            else:
                ## TODO can I rewrite it as multiprocessing?
                Parallel(n_jobs=6, require="sharedmem")(
                    delayed(self._get_labels_features_edge_weights_seizure)(patient)
                    for patient in patient_list
                )
                Parallel(n_jobs=6, require="sharedmem")(
                delayed(self._get_labels_features_edge_weights_interictal)(patient)
                for patient in patient_list
                )    
            self.hdf5_file.close()
            print(f"Dataset created in folder {dataset_folder}.")
            print(f"Dataset contains {self.sample_count} samples.")
        except:
            self.hdf5_file.close()
            os.remove(dataset_path)
            os.remove(os.path.join(dataset_folder, "config.yaml"))
            os.rmdir(dataset_folder)
            raise Exception("Dataset creation failed. Dataset deleted.")

In [ ]:
torch_geometric.seed_everything(42)
hdf_writer = HDFDataset_Writer(
    npy_dataset_path="data/npy_data_full",
    event_tables_path="data/event_tables",
    cache_folder="cache",
    seizure_lookback=600,
    sample_timestep=9,
    downsample=60,
    connectivity_metric="corr"
)

In [ ]:
# torch_geometric.seed_everything(42)
# hdf_writer.get_dataset()

In [56]:
class HDFDatasetLoader:
    """Class to load graph data from HDF5 file as lists of torch.geomtric.data.Data objects."""

    def __init__(
        self,
        root,
        train_val_split_ratio: float = 0.0,
        loso_subject: str = None,
        sampling_f : int = 60,
        hjorth: bool = False,
        fft: bool = False,
        seed: int = 42,
    ) -> None:
        assert not (hjorth and fft), "Cannot use both hjorth and fft feature conversion!"
        self.hdf_data_path = f"{root}/dataset.hdf5"
        self.hdf_dataset = h5py.File(self.hdf_data_path, "r")
        self.patient_list = list(self.hdf_dataset.keys())
        self.loso_subject = loso_subject
        self.train_val_split_ratio = train_val_split_ratio
        self.seed = seed
        self.hjorth = hjorth
        self.fft = fft
        self.sampling_f = sampling_f
        if loso_subject is not None:
            self.patient_list.remove(loso_subject)

        self._determine_sample_count()
        self._determine_n_channels()
        self._determine_n_features()

    def _determine_sample_count(self):
        total_samples = 0
        for patient in self.patient_list:
            total_samples += self.hdf_dataset[patient]["features"].shape[0]
        if self.train_val_split_ratio > 0:
            self.train_samples = int(total_samples * (1 - self.train_val_split_ratio))
            self.val_samples = total_samples - self.train_samples
        else:
            self.train_samples = total_samples
            self.val_samples = 0
        if self.loso_subject is not None:
            self.loso_samples = self.hdf_dataset[self.loso_subject]["features"].shape[0]
        else:
            self.loso_samples = 0

    def _determine_n_channels(self):
        """Get the number of channels in the dataset"""
        self.n_channels = self.hdf_dataset[self.patient_list[0]]["features"].shape[1]

    def _determine_n_features(self):
        """Get the number of features for every sample in the dataset."""
        self.n_features = self.hdf_dataset[self.patient_list[0]]["features"].shape[2]

    def _get_mean_std(self):
        """Method to deterimen mean and standard deviation of interictal samples. Those values are used late to normalize all data."""
        current_sample = 0
        features_all = np.empty(
            (self.train_samples + self.val_samples, self.n_channels, self.n_features)
        )  ## for standarization only
        labels_all = np.empty((self.train_samples + self.val_samples, 1))
        for patient in self.patient_list:
            features_all[
                current_sample : current_sample
                + self.hdf_dataset[patient]["features"].shape[0]
            ] = self.hdf_dataset[patient]["features"]
            labels_all[
                current_sample : current_sample
                + self.hdf_dataset[patient]["features"].shape[0]
            ] = self.hdf_dataset[patient]["labels"]
            current_sample += self.hdf_dataset[patient]["features"].shape[0]
        interictal_idx = np.where(labels_all == 2)[0]
        interictal_mean = np.mean(features_all[interictal_idx])
        interictal_std = np.std(features_all[interictal_idx])

        self.mean_interictal = interictal_mean
        self.std_interictal = interictal_std

    def _calculate_hjorth_features(self, features):
        """Converting features to Hjorth features.
        Args:
            features: (np.ndarray) Array with features to be converted.
        Returns:
            new_features: (np.ndarray) Array with Hjorth features.
        """
        FREQ_BANDS = [0.5, 4, 8, 13, 29]  ## hardcoded for now
        band_count = len(FREQ_BANDS) - 1
       
        new_features = np.array(
            [
                np.concatenate(
                    [
                        np.expand_dims(compute_variance(feature), 1),
                        np.expand_dims(compute_hjorth_mobility(feature), 1),
                        np.expand_dims(compute_hjorth_complexity(feature), 1),
                        np.expand_dims(compute_line_length(feature), 1),
                        np.expand_dims(compute_katz_fd(feature), 1),
                        np.expand_dims(compute_higuchi_fd(feature), 1),
                        compute_energy_freq_bands(
                            self.sampling_f, feature, FREQ_BANDS
                        ).reshape(self.n_channels, band_count),
                    ],
                    axis=1,
                )
                for feature in features
            ]
        )
        return new_features

    def _get_train_val_indices(self, n_samples,labels):
        
        train_indices, val_indices = train_test_split(
            n_samples,
            test_size=self.train_val_split_ratio,
            shuffle=True,
            stratify=labels,
            random_state=self.seed,
        )
        train_indices = np.sort(train_indices)
        val_indices = np.sort(val_indices)
        return train_indices, val_indices

    def _transform_features(self, features):
        processed_features = (features - self.mean_interictal) / self.std_interictal
        if self.hjorth:
            processed_features = self._calculate_hjorth_features(processed_features)
        elif self.fft:
            processed_features = np.fft.rfft(processed_features)
        return processed_features

    def _features_to_data_list(self, features, edge_idx, labels):
        """Converts features, edges and labels to list of torch_geometric.data.Data objects.
        Before the conversion, features are normalized using mean and standard deviation of interictal samples
        and transformed if specified in the config.
        Args:
            features: (np.ndarray) Array with features.
            edges: (np.ndarray) Array with edges.
            labels: (np.ndarray) Array with labels.
        Returns:
            data_list: (list) List of torch_geometric.data.Data objects.
        """
        processed_features = self._transform_features(features)
        data_list = [
            Data(
                x=processed_features[i],
                edge_index=edge_idx[i],
                y=labels[i],
            )
            for i in range(len(processed_features))
        ]
        return data_list

    def _get_loso_data(self):
        """Get data for leave-one-subject-out cross-validation.
        Returns:
            loso_data_list: (list) List of torch_geometric.data.Data objects for leave-one-subject-out cross-validation.
        """
        hdf5_file = h5py.File(self.hdf_data_path, "r")
        loso_features = hdf5_file[self.loso_subject]["features"]
        loso_labels = hdf5_file[self.loso_subject]["labels"]
        loso_edge_idx = hdf5_file[self.loso_subject]["edge_idx"]
        loso_data_list = self._features_to_data_list(
            loso_features, loso_edge_idx, loso_labels
        )
        hdf5_file.close()
        return loso_data_list

    def _get_train_val_data(self, patient):
        """Get patient data for train and validation sets.
        Args:
            patient: (str) Name of the patient to get the data for.
        Returns:
            train_data_list: (list) List of torch_geometric.data.Data objects for training.
            val_data_list: (list) List of torch_geometric.data.Data objects for validation.
        """
        hdf5_file = h5py.File(self.hdf_data_path, "r")
        n_samples = np.arange(hdf5_file[patient]["features"].shape[0])
        labels = np.squeeze(hdf5_file[patient]["labels"])
        
        train_indices, val_indices = self._get_train_val_indices(n_samples,labels)
        features_train = hdf5_file[patient]["features"][train_indices]

        labels_train = hdf5_file[patient]["labels"][train_indices]
        edge_idx_train = hdf5_file[patient]["edge_idx"][train_indices]

        features_val = hdf5_file[patient]["features"][val_indices]

        labels_val = hdf5_file[patient]["labels"][val_indices]
        edge_idx_val = hdf5_file[patient]["edge_idx"][val_indices]

        data_list_train = self._features_to_data_list(
            features_train, edge_idx_train, labels_train
        )
        data_list_val = self._features_to_data_list(
            features_val, edge_idx_val, labels_val
        )
        hdf5_file.close()
        return (data_list_train, data_list_val)

    def _get_train_data(self, patient):
        """Get patient data for training set.
        Args:
            patient: (str) Name of the patient to get the data for.
        Returns:
            train_data_list: (list) List of torch_geometric.data.Data objects for training.
        """
        hdf5_file = h5py.File(self.hdf_data_path, "r")
        features_train = hdf5_file[patient]["features"]
        labels_train = hdf5_file[patient]["labels"]
        edge_idx_train = hdf5_file[patient]["edge_idx"]
        train_data_list = self._features_to_data_list(
            features_train, edge_idx_train, labels_train
        )
        hdf5_file.close()        

        return train_data_list
    def get_data(self):
        """Get data for training, validation and leave-one-subject-out cross-validation.
        Returns:
            data_lists: (list) List of lists of torch_geometric.data.Data objects loaded.
        """
        train_data_list = []
        if self.train_val_split_ratio > 0:
            val_data_list = []
        self._get_mean_std()
        # result_list = Parallel(n_jobs=6, require="sharedmem")(
        #     delayed(self._get_train_val_data)(patient)
        #     for patient in self.patient_list
        # )
        ## TODO we cannot have hdf_dataset (even closed) as an attribute of a class here - make it 
        ## more elegant
        
        ## TODO check if the results are the same (but why they should not be?)
        self.hdf_dataset.close()
        del self.hdf_dataset
        gc.collect()
        
        num_processes = mp.cpu_count()
        pool = mp.Pool(processes=num_processes)
        result_list = pool.map(self._get_train_val_data, self.patient_list)
        pool.close()
        pool.join()
        for train_data, val_data in result_list:
            train_data_list = train_data_list + train_data
            val_data_list = val_data_list + val_data
        # for patient in self.patient_list:
        #     if self.train_val_split_ratio > 0:
        #         train_list, val_list = self._get_train_val_data(patient)
        #         train_data_list = train_data_list + train_list
        #         val_data_list = val_data_list + val_list
        #     else:
        #         train_list = self._get_train_data(patient)
        #         train_data_list = train_data_list + train_list
 
        data_lists = (
            [train_data_list]
            if self.train_val_split_ratio == 0
            else [train_data_list, val_data_list]
        )
        if self.loso_subject is not None:
            loso_data_list = self._get_loso_data()
            data_lists.append(loso_data_list)

        return data_lists

In [57]:
from time import time
start = time()
root = "cache/2023-07-03_15-23-17"
dataset = HDFDatasetLoader(root, train_val_split_ratio=0.2, loso_subject="chb01", hjorth=True, fft=False)
data_lists = dataset.get_data()
print(f"Time elapsed: {time() - start}")

Time elapsed: 143.37564492225647


In [22]:
import multiprocessing as mp
mp.cpu_count()

8

In [4]:
root = "cache/2023-07-03_15-23-17"

In [35]:
import time

In [28]:
def read_patient_data(patient_id):
    root = "cache/2023-07-03_15-23-17"
    file_path = f"{root}/dataset.hdf5"
    file = h5py.File(file_path, 'r')
    dataset = file[patient_id]['features']
    data = dataset[()]  # Read the entire dataset
    file.close()
    time.sleep(10)
    return data

In [37]:
start = time.time()
hdf5_file = h5py.File(f"{root}/dataset.hdf5", "r")
patient_list = list(hdf5_file.keys())
hdf5_file.close()

num_processes = 6
pool = mp.Pool(processes=num_processes)
results = pool.map(read_patient_data, patient_list)
pool.close()
pool.join()
print(f"Time taken: {time.time()-start}")

KeyboardInterrupt: 

In [25]:
start = time.time()
features_list = []
hdf5_file = h5py.File(f"{root}/dataset.hdf5", "r")
for patient in patient_list:
    features_list.append(hdf5_file[patient]["features"][()])
    time.sleep(10)
hdf5_file.close()
print(f"Time taken: {time.time()-start}")

Time taken: 241.33937168121338


In [ ]:
### CHECK HOW THE POWER IS CALCULATED!!!

In [ ]:
FREQ_BANDS = [0.5, 4]  ## hardcoded for now
band_count = len(FREQ_BANDS) - 1
bands_mne = np.stack([compute_energy_freq_bands(60, feature, FREQ_BANDS).reshape(18, band_count) for feature in patient_1_samples])

In [ ]:
def calculate_energy(signal, sample_rate, frequency_bands):
    # Apply FFT to the signal
    fft_result = np.fft.fft(signal, axis=2)
    
    # Compute the power spectrum
    power_spectrum = np.abs(fft_result) ** 2
    
    # Get the frequency axis
    frequencies = np.fft.fftfreq(signal.shape[2], d=1/sample_rate)
    
    # Initialize dictionary to store energy for each frequency band
    energy = {band: [] for band in frequency_bands}
    
    # Calculate energy for each sample in the matrix
    for sample in power_spectrum:
        for band in frequency_bands:
            # Find the indices corresponding to the frequency band
            band_indices = np.where((frequencies >= band[0]) & (frequencies <= band[1]))[0]
            
            # Sum the power spectrum values within the frequency band
            band_energy = np.sum(sample[..., band_indices], axis=-1)
            
            # Append the energy to the corresponding band in the dictionary
            energy[band].append(band_energy)
    
    # Convert energy lists to NumPy arrays
    for band in energy:
        energy[band] = np.array(energy[band])
    
    return energy

# Example usage
# Assuming you have your [N, 18, 540] signal stored in 'eeg_signal' variable
sample_rate = 60  # Specify the sample rate of your EEG signal
frequency_bands = {
    'delta': (0.5, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 29),
}

energy = calculate_energy(patient_1_samples, sample_rate, frequency_bands)

In [ ]:
import numpy as np

# Assuming your data matrix is called 'eeg_data' with shape (N, 18, 540)

# Step 1: Apply Fourier Transform along the time axis
fft_data = np.fft.fft(patient_1_samples, axis=2)

# Step 2: Calculate power spectrum
power_spectrum = np.abs(fft_data)**2

# Step 3: Define frequency bands of interest
frequency_bands = {
    'delta': (0.5, 4),    # Delta band (0.5-4 Hz)
    # 'theta': (4, 8),      # Theta band (4-8 Hz)
    # 'alpha': (8, 13),     # Alpha band (8-13 Hz)
    # 'beta': (13, 29),     # Beta band (13-30 Hz)
}
freq = np.fft.fftfreq(540, d=1/60)
# Calculate power within each frequency band for each sample
power_bands = {}
for band_name, (f_min, f_max) in frequency_bands.items():
    freq_indices = np.where((f_min <= freq) & (freq <= f_max))
    power_bands[band_name] = np.expand_dims(np.mean(power_spectrum[:, :, freq_indices[0]], axis=(2)), axis=2)

# Now you have the average power within each frequency band for each sample in 'power_bands'
# The shape of each power band will be (N, 18)


In [ ]:
bands_mne

In [ ]:
power_bands['delta'].mean()

In [ ]:
np.where((f_min <= freq) & (freq <= f_max))[0]

In [ ]:
freq_indices

In [ ]:
power_spectrum[:, :, freq_indices].shape

In [ ]:
power_spectrum.shape

In [ ]:
freq_indices

In [ ]:
dataset = torch.load(root + "/processed/dataset.pt")